In [ ]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("jobs_schema", "")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
jobs_schema = dbutils.widgets.get("jobs_schema")

if not catalog or not schema or not jobs_schema:
    raise ValueError("Missing required widget: catalog, schema, or jobs_schema")

agg_table = f"{catalog}.{schema}.go_property_valuations"
update_table = f"{jobs_schema}.updated_valuations"

class ValuationUpdateProcessor:
    def __init__(self, spark: SparkSession, agg_table: str, update_table: str):
        self.spark = spark
        self.agg_table = agg_table
        self.update_table = update_table

    def recalculate_valuation_metrics(self, agg_df: DataFrame) -> DataFrame:
        return (
            agg_df.withColumn(
                "adjusted_sale_price_eur",
                F.round(F.col("avg_sale_price_eur") * (1 + (F.lit(0.05) * F.rand())), 2)
            ).withColumn("update_timestamp", F.current_timestamp())
        )

    def save_valuation_updates(self, df: DataFrame) -> None:
        df.write.format("delta").mode("append").saveAsTable(self.update_table)

processor = ValuationUpdateProcessor(spark, agg_table, update_table)
agg_df = spark.table(agg_table)
updated_df = processor.recalculate_valuation_metrics(agg_df)
processor.save_valuation_updates(updated_df)